In [2]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt

In [156]:
bins

{'bin0': ['2021-03-07 15:00', '2021-03-08 09:30'],
 'bin1': ['2021-03-08 09:30', '2021-03-08 09:39'],
 'bin2': ['2021-03-08 09:39', '2021-03-08 09:48'],
 'bin3': ['2021-03-08 09:48', '2021-03-08 09:57'],
 'bin4': ['2021-03-08 09:57', '2021-03-08 10:06'],
 'bin5': ['2021-03-08 10:06', '2021-03-08 10:15'],
 'bin6': ['2021-03-08 10:15', '2021-03-08 10:24'],
 'bin7': ['2021-03-08 10:24', '2021-03-08 10:33'],
 'bin8': ['2021-03-08 10:33', '2021-03-08 10:42'],
 'bin9': ['2021-03-08 10:42', '2021-03-08 10:51'],
 'bin10': ['2021-03-08 10:51', '2021-03-08 11:30'],
 'bin11': ['2021-03-08 11:30', '2021-03-08 13:00'],
 'bin12': ['2021-03-08 13:00', '2021-03-08 13:09'],
 'bin13': ['2021-03-08 13:09', '2021-03-08 13:18'],
 'bin14': ['2021-03-08 13:18', '2021-03-08 13:27'],
 'bin15': ['2021-03-08 13:27', '2021-03-08 13:36'],
 'bin16': ['2021-03-08 13:36', '2021-03-08 13:45'],
 'bin17': ['2021-03-08 13:45', '2021-03-08 13:54'],
 'bin18': ['2021-03-08 13:54', '2021-03-08 14:03'],
 'bin19': ['2021-03-08

In [23]:
from BuildData import BuildData

conf = {'lag_day': 3,
        'lag_bin': 3,
        'lag_week': 1,
        'bin_num': 24,
        'file_dir': '../data/0308/0308-data/',
        'comment_dir': '../data/0308/0308-number/'}

# inputs_df, output_list = BuildData(conf= conf).gen_input_output_data(file_path= '../data/0308/0308-data/000046_XSHE_25_daily.csv',
#                                                                      comment_path= '../data/0308/0308-number/000046_comment_sentiment.csv',
#                                                                      stock_info= None)

path = '../data/volume/0308/Input/000046_3_3_inputs.npy'
input_ = np.load('../data/volume/0308/Input/000046_3_3_inputs.npy', allow_pickle= True)
output_matrix = np.load(path.replace('inputs', 'output').replace('Input', 'Output'), allow_pickle= True)
input_matrix = np.array([value for item in input_ for value in item], dtype= np.float32).reshape(input_.shape[0], 13, 9)

In [53]:
'''
Author: h-jie huangjie20011001@163.com
Date: 2024-06-23 16:19:53
'''
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GraphAttentionLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout= 0.5, alpha= 0.2, concat= True, device= device, *args, **kwargs) -> None:
        super(GraphAttentionLayer, self).__init__(*args, **kwargs)
        self.in_fetures = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha
        self.concat = True

        self.device = device
        self.W = nn.Parameter(torch.empty(size= (in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)  # 初始化
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def MLP(self, N):
        self.a = nn.Parameter(torch.empty(size= (2*self.out_features, N))).to(device= device) # 2FxN
        nn.init.xavier_uniform_(self.a.data, gain= 1.414)

    def _prepare_attentional_mechanism_input(self, h):
        """
        infer: 
            https://github.com/Diego999/pyGAT/blob/master/layers.py
        """
        h1 = torch.matmul(h, self.a[:self.out_features, :])
        h2 = torch.matmul(h, self.a[self.out_features:, :])
        # broadcast add
        # print(h1.shape, h2.shape)
        e = h1 + h2
        return self.leakyrelu(e)
    
    def forward(self, inp, adj):
        """
        inp： input_features [B, N, in_features]
        adj: adjacent_matrix [N, N]
        """
        h = torch.matmul(inp, self.W) # 计算 w*x
        adj = adj.to(h.device)
        N = h.size()[1] # TODO: 对于不同的需要需改，对于batch_size 修改为2 对应节点数量
        self.MLP(N)

        e = self._prepare_attentional_mechanism_input(h)
        zero_vec = -1e12 * torch.ones_like(e).to(h.device)   # 将没有连接的边置为负无穷

        attention = torch.where(adj> 0, e, zero_vec)   # [B, N, N]
        attention = F.softmax(attention, dim=1)    # [B, N, N]！
        attention = F.dropout(attention, self.dropout)
        h_prime = torch.matmul(attention, h)  # [B, N, N].[B, N, out_features] => [B, N, out_features]
        if self.concat:
            return F.relu(h_prime)
        else:
            return h_prime 

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'
    
class GAT(nn.Module):

    def __init__(self, n_feat, n_hid, n_class, dropout:float=0.5, alpha: float=0.3, n_heads: int=3, *args, **kwargs) -> None:
        super(GAT, self).__init__(*args, **kwargs)
        self.dropout = dropout 

        self.attentions = [GraphAttentionLayer(n_feat, n_hid, dropout=dropout, alpha=alpha, concat=True) for _ in range(n_heads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)   
        self.out_att = GraphAttentionLayer(n_hid * n_heads, n_class, dropout=dropout,alpha=alpha, concat=False)
    
    def forward(self, x, adj):
        x = F.dropout(x, self.dropout)  
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        print('多头注意力结果输出：', x.shape) 
        x = F.dropout(x, self.dropout)

        # x = self.out_att(x, adj)
        print('3', x.shape)
        x = F.relu(x)
        return F.softmax(x, dim= 1)

if __name__ == "__main__":
    # input: batch length node features
    # output: batch length features
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    x = torch.rand(size= (32, 10, 13, 7)).to(device= device)
    x = x.permute(0, 2, 1, 3)
    x = x.reshape(32, 13, -1)
    print(x.shape)
    adj_matrix = torch.rand(size= (13, 13))

    model = GAT(n_feat= 70, n_hid= 14, n_class= 7, n_heads= 2).to(device= device) # n_class 代表未来预测的日期
    out = model(x, adj_matrix)
    print('输出', out.shape)

torch.Size([32, 13, 70])
多头注意力结果输出： torch.Size([32, 26, 14])
3 torch.Size([32, 26, 14])
输出 torch.Size([32, 26, 14])


In [51]:
x = torch.randn(32, 13, 7)
print(x.shape)

torch.Size([32, 13, 7])


In [55]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GraphAttentionLayer(nn.Module):

    def __init__(self, in_features, out_features, dropout= 0.5, alpha= 0.2, concat= True, device= device, *args, **kwargs) -> None:
        super(GraphAttentionLayer, self).__init__(*args, **kwargs)
        self.in_fetures = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha
        self.concat = True

        self.device = device

        self.W = nn.Parameter(torch.empty(size= (in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)  # 初始化
        
        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def MLP(self, N):
        self.a = nn.Parameter(torch.empty(size= (2*self.out_features, N))).to(device= device) # 2FxN 
        nn.init.xavier_uniform_(self.a.data, gain= 1.414)

    def _prepare_attentional_mechanism_input(self, h):
        """
        infer: 
            https://github.com/Diego999/pyGAT/blob/master/layers.py
        """
        h1 = torch.matmul(h, self.a[:self.out_features, :])
        h2 = torch.matmul(h, self.a[self.out_features:, :])
        # broadcast add
        e = h1 + h2
        return self.leakyrelu(e)
    
    def forward(self, inp, adj):
        """
        inp： input_features [B, N, in_features]
        adj: adjacent_matrix [N, N]
        """
        h = torch.matmul(inp, self.W) # 计算 w*x
        adj = adj.to(h.device)
        N = h.size()[1]
        self.MLP(N)

        e = self._prepare_attentional_mechanism_input(h)
        zero_vec = -1e12 * torch.ones_like(e).to(h.device)   # 将没有连接的边置为负无穷

        attention = torch.where(adj> 0, e, zero_vec)   # [B, N, N]
        attention = F.softmax(attention, dim=1)    # [B, N, N]！
        attention = F.dropout(attention, self.dropout)
        h_prime = torch.matmul(attention, h)  # [B, N, N].[B, N, out_features] => [B, N, out_features]
        if self.concat:
            return F.relu(h_prime)
        else:
            return h_prime 
    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'
    
class GAT(nn.Module):

    def __init__(self, n_feat, n_hid, n_class, dropout:float=0.5, alpha: float=0.3, n_heads: int=3, *args, **kwargs) -> None:
        super(GAT, self).__init__(*args, **kwargs)
        self.dropout = dropout 

        self.attentions = [GraphAttentionLayer(n_feat, n_hid, dropout=dropout, alpha=alpha, concat=True) for _ in range(n_heads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)   
        self.out_att = GraphAttentionLayer(n_hid * n_heads, n_class, dropout=dropout,alpha=alpha, concat=False)
    
    def forward(self, x, adj):
        x = F.dropout(x, self.dropout)  
        x = torch.cat([att(x, adj) for att in self.attentions], dim=2) 
        x = F.dropout(x, self.dropout)  
        x = self.out_att(x, adj)
        
        #x = F.log_softmax(x, dim=2)[:, -1, :]
        return x[:, -1, :] # log_softmax速度变快，保持数值稳定

if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    x = torch.rand(size= (10, 13, 7)).to(device= device)
    adj_matrix = torch.rand(size= (13, 13))

    model = GAT(n_feat= 7, n_hid= 14, n_class= 7, n_heads= 2).to(device= device) # n_class 代表未来预测的日期
    out = model(x, adj_matrix)
    print(out.shape)

torch.Size([10, 7])
